In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd

browser = Browser('chrome')

In [ ]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]"
browser.visit(url)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [ ]:
# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)


In [4]:
cookie = browser.find_by_id('btn-cookie-allow').click()
# cookie.click()

In [5]:
# This is for loading more products 
max_iteration = 1
iteration = 0

while iteration<= max_iteration:
    # Extract the HTML of the current page
    html = browser.html
    
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')
    
    all_links = soup.find_all("div", class_ = 'coveo-product-items')

    links = []
    
    for link in all_links:
        results = link.a['href']
        links.append(results)
        
    # Find the element by ID
    element = soup.find(id='loadMore')
    
    if element is None:
        # Element not found, exit the loop
        break
        
    # Wait for 2 seconds
    time.sleep(1)
    
    
        
    # Click on the element
    browser.find_by_id('loadMore').click()
    
    
    
    iteration += 1
    
    print(f'Iteration #: {iteration}')
    


Iteration #: 1
Iteration #: 2


In [6]:
len(links)

48

In [8]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []

max_iteration2 = 4
iteration2 = 0

while iteration2 <= max_iteration2:
    for record in links:
        url = record
        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
    #     results = browser.links.find_by_text('English')
    #     results.click()

        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)

        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)

        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
        try:
            onsale.append(soup.find('div', class_ = 'amlabel-text').text)
        except AttributeError:
            rating.append('N/A')
        try:
            rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
        except AttributeError:
            rating.append('N/A')
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
        description.append(soup.find('div', class_ = 'testing_note').text)

        categories_all = soup.find('div', class_ = 'breadcrumbs')
        categories = categories_all.find_all('li', class_ = 'item')
        category_list = []

        for category_record in categories:
            category_list.append(category_record.text)
        category = category_list[-2]

        moredetail = soup.find('div', class_ = 'moredetail')
        all_details = moredetail.find_all('li')
        details = {}
        
        for detail_record in all_details:
            details_value = detail_record.find('div', class_ = 'value').text
            details_name = detail_record.find('div', class_ = 'label').text
            details[details_name] = details_value
        
        details_list.append(details)
        
        print(iteration2)
        iteration2 += 1
        
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [9]:
# old_price = soup.find('span', class_ = 'old-price')
# old = old_price.find('span', class_ = 'price')
# regular_price = old.find('span', class_ = 'price').text

# sale_price = soup.find('span', class_ = 'special-price')
# new_price = sale_price.find('span', class_ = 'price')
# onsale = new_price.find('span', class_ = 'price').text

# name = soup.find('div', class_ = 'page-title-wrapper').text
# onsale = soup.find('div', class_ = 'amlabel-text').text
# rating = soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text
# reviews = soup.find('div', class_ = 'bv_numReviews_text').text
# size = soup.find('div', class_ = 'lcbo-product-size').text
# description = soup.find('div', class_ = 'testing_note').text

# categories_all = soup.find('div', class_ = 'breadcrumbs')
# categories = categories_all.find_all('li', class_ = 'item')
# category_list = []
# for record in categories:
#     category_list.append(record.text)
# category = category_list[-2]

# moredetail = soup.find('div', class_ = 'moredetail')
# all_details = moredetail.find_all('li')
# len_details = len(all_details)
# details = {}

# for record in all_details:
#     details_value = record.find('div', class_ = 'value').text
#     details_name = record.find('div', class_ = 'label').text
#     details[details_name] = details_value

In [13]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category
}

# regular_price = [] ok
# sale_price = [] ok
# name = [] ok
# rating = [] ok
# reviews = [] ok
# size = [] ok
# description = [] ok
# category = [] ok 

# details = {}
# Add the details dictionary to the data dictionary
data['Details'] = details_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

details_df = pd.json_normalize(df['Details'])

df = pd.concat([df, details_df], axis = 1)

df.drop('Details', axis = 1, inplace = True)

# Display the dataframe Test
df.head(20)

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Alcohol/Vol,Made In,By,Sugar Content,Varietal,Release Date
0,Ruffino Prosecco Rose DOC,$20.00,$18.00,5.0,(2),750 ml bottle,Ruffino has been at the forefront of Italian w...,Red Wine,11%,"Friuli, Italy",Ruffino Srl Wines,21 g/L,Sparkling,NaN
1,Josh Cellars Pinot Grigio,$19.95,$17.95,4.5,(17),750 ml bottle,Josh Cellars is a renowned brand created to ce...,Red Wine,12.5%,"California, United States",Josh Cellars,3 g/L,Pinot Grigio,NaN
2,Collavini Pinot Grigio,$16.55,$14.55,4.5,(6),750 ml bottle,"This pinot grigio is a customer favourite, and...",Red Wine,12.5%,"Veneto, Italy",Collavini,4 g/L,Pinot Grigio,NaN
3,Nude Pink Lemonade,$2.85,$2.70,3.9,(16),355 ml can,"Nude's newest seltzer is an all-natural, prett...",Red Wine,5%,"Ontario, Canada",Nude,NaN,NaN,NaN
4,Meukow Cognac VS,$65.95,$61.95,4.7,(3),750 ml bottle,Awarded in 2015 both Best Cognac VS and Double...,Red Wine,40%,"Cognac, France",Meukow,NaN,NaN,NaN
5,Herradura Reposado Tequila,$75.20,$70.20,4.7,(94),750 ml bottle,Handcrafted tequila that is created for sippin...,Red Wine,40%,Mexico,Herradura,NaN,NaN,NaN
6,Santa Margherita Brut Valdobbiadene Prosecco ...,$20.95,$18.95,4.8,(12),750 ml bottle,We've heard this Prosecco is a celebrity favou...,Red Wine,11.5%,"Veneto, Italy",Santa Margherita,13 g/L,Sparkling - Dry,"April 1, 2022"
7,Old Style Pilsner,$2.25,$2.05,4.0,(1),473 ml can,Crafted in the Pilsner style using the same re...,Red Wine,5%,"Ontario, Canada",NaN,NaN,NaN,NaN
8,Pelee Island Best in Show Pinot Grigio,$11.95,$9.95,3.0,(2),750 ml bottle,No wonder this winery is the best in show - it...,Red Wine,12.5%,Canada,Pelee Island Winery,10 g/L,Pinot Grigio,NaN
9,Coors Light,$16.95,$14.95,N/A,(0),6 x 473 ml can,Pale gold colour; light grain with a touch of ...,Red Wine,4%,"Ontario, Canada",Coors,NaN,NaN,NaN


In [ ]:
category_list

In [ ]:
categories_all = soup.find('div', class_ = 'breadcrumbs')
categories = categories_all.find_all('li', class_ = 'item')

category_list1 = []
    
for category_record in categories:
        category_list1.append(category_record.text)
        category_list1